In [3]:
import pandas as pd

#Naibe Baise
#SVM
#Xg boost
#logistic_regression

train_df = pd.read_csv('/Users/azaruddin.sk/Desktop/YUbi/NLP Task/train.tsv', sep='\t')
test_df = pd.read_csv('/Users/azaruddin.sk/Desktop/YUbi/NLP Task/test.tsv', sep='\t')

#print(train_df.isnull().sum())
#print(test_df.isnull().sum())
train_df = train_df.drop(columns=['Sector'])
test_df = test_df.drop(columns=['Sector'])
train_df['News Title'].fillna('', inplace=True)
train_df['News Description'].fillna('', inplace=True)
test_df['News Title'].fillna('', inplace=True)
test_df['News Description'].fillna('', inplace=True)

train_df['text'] = train_df['News Title'] + " " + train_df['News Description']
test_df['text'] = test_df['News Title'] + " " + test_df['News Description']

In [2]:
from nltk.tokenize import RegexpTokenizer  #tokenizer library regexptokenizer to remove punctuation
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = list(set(nltk.corpus.stopwords.words('english'))) #decalred stop_words as variable for getting stopwords from english langaugae & added set() function to remove duplicate stop_words
lemmatizer = WordNetLemmatizer()

def text_preprocess(input_text):
    if not isinstance(input_text, str):
        return ""
    input_text = input_text.lower()
    input_text = input_text.replace("\n"," ")
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input_text)
    clean_tokens = []
    for token in tokens:
        if (token not in stop_words) and (len(token) >2):
            token = lemmatizer.lemmatize(token)
            clean_tokens.append(token)
    return " ".join(clean_tokens)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/azaruddin.sk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/azaruddin.sk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
clean_text = []
for text in train_df['text'].values:
    cl_txt = text_preprocess(text)
    clean_text.append(cl_txt)
train_df['clean_text'] = clean_text


pre_text = []
for text in test_df['text'].values:
    cl_txt = text_preprocess(text)
    pre_text.append(cl_txt)
test_df['clean_text'] = pre_text

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_df['clean_text'])
X_test = vectorizer.transform(test_df['clean_text'])

In [7]:
#Logistic Regressioin


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train_text, X_val_text, y_train, y_val = train_test_split(X_train, train_df['Sentiment'], test_size=0.2, random_state=42)

model_1 = LogisticRegression(max_iter=1000)
model_1.fit(X_train_text, y_train)

y_val_pred = model_1.predict(X_val_text)
print(classification_report(y_val, y_val_pred))

y_test_pred = model_1.predict(X_test)


submission = pd.DataFrame({
    'News ID': test_df['News ID'],
    'Sentiment': y_test_pred
})

file_path = '/Users/azaruddin.sk/Desktop/YUbi/NLP Task/submission_Logistic_re.csv'

submission.to_csv(file_path, index=False)

              precision    recall  f1-score   support

       mixed       0.65      0.14      0.22       412
    negative       0.69      0.49      0.57      3195
     neutral       0.86      0.92      0.89     26044
    positive       0.69      0.61      0.65      6699

    accuracy                           0.82     36350
   macro avg       0.72      0.54      0.58     36350
weighted avg       0.81      0.82      0.81     36350



In [8]:
#Naive Bayse

from sklearn.naive_bayes import MultinomialNB
model_2 = MultinomialNB()
model_2.fit(X_train_text, y_train)

val_predictions = model_2.predict(X_val_text)

#val_accuracy = accuracy_score(y_val, val_predictions)
#print(f"Validation Accuracy: {val_accuracy:.4f}")

print(classification_report(y_val, val_predictions))

test_predictions = model_2.predict(X_test)

submission = pd.DataFrame({
    'News ID': test_df['News ID'],
    'Sentiment': test_predictions
})

file_path = '/Users/azaruddin.sk/Desktop/YUbi/NLP Task/submission_NB.csv'

submission.to_csv(file_path, index=False)

              precision    recall  f1-score   support

       mixed       0.62      0.02      0.04       412
    negative       0.61      0.39      0.47      3195
     neutral       0.85      0.90      0.87     26044
    positive       0.61      0.61      0.61      6699

    accuracy                           0.79     36350
   macro avg       0.67      0.48      0.50     36350
weighted avg       0.78      0.79      0.78     36350



In [9]:
# SVM model

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

model_3 = SVC(kernel='linear', C=1.0)  # You can tune the C parameter if needed
model_3.fit(X_train_text, y_train)

val_predictions_SVC = model_3.predict(X_val_text)
print(classification_report(y_val, val_predictions_SVC))

test_predictions_SVM = model_3.predict(X_test)

submission = pd.DataFrame({
    'News ID': test_df['News ID'],
    'Sentiment': test_predictions_SVM
})

file_path = '/Users/azaruddin.sk/Desktop/YUbi/NLP Task/submission_SVM.csv'

submission.to_csv(file_path, index=False)

              precision    recall  f1-score   support

       mixed       0.62      0.16      0.25       412
    negative       0.69      0.49      0.57      3195
     neutral       0.86      0.92      0.89     26044
    positive       0.70      0.63      0.66      6699

    accuracy                           0.82     36350
   macro avg       0.72      0.55      0.59     36350
weighted avg       0.81      0.82      0.81     36350

